In [1]:
import pandas as pd
import numpy as np

datasets = ['imdb', 'tpch', 'ssb']
dfs = []


for d in datasets:
    dfs.append(pd.read_csv(f"../data/{d}/{d}_all.csv"))

ds = pd.concat(dfs)
ds

,left_cardinality,left_cardinality_ratio,base_cardinality,selectivity_on_indexed_attr,left_ordered,base_ordered,hj_idx_cost,hj_seq_cost,nl_idx_cost,nl_seq_cost,mj_idx_cost,mj_seq_cost,optimal_decision
0,305712,0.008435,36244344,0.770287,0,0,11882673.70,11627049.47,2.038630e+11,2.038630e+11,15974923.66,15719299.43,3
1,963396,0.026581,36244344,0.938884,0,0,15641358.87,15145064.13,7.867570e+11,7.867560e+11,18233675.75,17737381.01,3
2,2219732,0.061244,36244344,0.585043,0,0,20769144.12,20771637.25,1.123490e+12,1.123490e+12,15553277.61,15555770.74,4
3,374565,0.010334,36244344,0.375608,0,0,11186868.01,11480590.10,1.216830e+11,1.216830e+11,12152898.95,12446621.04,2
4,736327,0.020316,36244344,0.921837,0,0,14449520.09,13978150.55,5.898990e+11,5.898980e+11,17874241.41,17402871.87,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,193,0.096500,2000,0.613500,1,1,4192205.93,4192198.18,4.192353e+06,4.195745e+06,4192210.27,4192268.54,3
7996,155,0.077500,2000,0.042000,1,1,4144685.19,4144733.31,4.144879e+06,4.144926e+06,4144690.42,4144740.57,2
7997,390,0.195000,2000,0.253500,1,1,4344434.38,4344460.66,4.344674e+06,4.347419e+06,4344448.01,4344496.91,2
7998,857,0.428500,2000,0.810500,1,1,4514916.31,4514895.58,4.515226e+06,4.535707e+06,4514950.84,4515012.66,3


# Classification

In [2]:
key_features = ['left_cardinality_ratio', 'selectivity_on_indexed_attr', 'left_ordered', 'base_ordered']
features = ['left_cardinality', 'left_cardinality_ratio', 'base_cardinality', 'selectivity_on_indexed_attr', 'left_ordered','base_ordered']
regression_targets = ['hj_idx_cost', 'hj_seq_cost', 'nl_idx_cost', 'nl_seq_cost','mj_idx_cost', 'mj_seq_cost']
classification_target = ['optimal_decision']


X = ds[features]
y = ds['optimal_decision']

from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

# scaler = preprocessing.StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

from interpret.glassbox import ExplainableBoostingClassifier

ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)

# print(f'Train accuracy: {clf.score(X_train, y_train)}')
# print(f'Test accuracy: {clf.score(X_test, y_test)}')

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/interpret/glassbox/ebm/ebm.py:723: UserWarning: Multiclass is still experimental. Subject to change per release.
  warn("Multiclass is still experimental. Subject to change per release.")


ExplainableBoostingClassifier(binning='quantile', early_stopping_rounds=50,
               early_stopping_tolerance=0,
               feature_names=['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5'],
               feature_types=['continuous', 'continuous', 'continuous', 'continuous', 'categorical', 'categorical'],
               inner_bags=0, interactions=0, learning_rate=0.01,
               mains='all', max_bins=255, max_leaves=3, max_rounds=5000,
               min_samples_leaf=2, n_jobs=-2, outer_bags=16,
               random_state=42, validation_size=0.15)

In [ ]:
np.sum(ebm.predict(X_test) == y_test) / len(y_test)